In [1]:
# ap1_v130.frm.part2.unxor.2 
# 000eff30 SOFTUNE REALOS/ARM is REALtime OS for ARM.COPYRIGHT(C) FUJITSU SEMICONDUCTOR LIMITED 1999
# 000f80d0 Shell Execute and RICOH
# 000fada0 H:/z01firm and 192.168.1.1, 255.255.255.0
# 000fe270 wlan/_ptpip_operation.c
# 001071e0 IPL Mount
# 0010ae20 RICOH Theta
# 0x0010ae9c-> 0x0010c816 Certificate bundle + private key
# RSA certificate
!dd if=firmwares/unxor/ap1_v130.frm.part2.unxor.2 of=firmwares/unxor/ap1_v130.frm.part2.rsa_key.pem skip=1093276 bs=1 count=6522
!openssl x509 -in firmwares/unxor/ap1_v130.frm.part2.rsa_key.pem -text| grep Subject:
!openssl x509 -in firmwares/unxor/ap1_v130.frm.part2.rsa_key.pem -text| grep -A 1 "Identifier" | tail -n 1

6522+0 enregistrements lus
6522+0 enregistrements écrits
6522 octets (6,5 kB) copiés, 0,0284737 s, 229 kB/s
        Subject: DC=local, DC=wifilabs, CN=Users, CN=wifi-user
                0B:C1:85:9B:11:BC:E9:6A:C0:4E:08:B3:87:BC:29:AF:35:6E:58:23


In [2]:
%run ThetaS.ipynb
# get best_class consensus_keys
# the firmware
firmwares = ["ap1_v110.frm", "ap1_v130.frm", "ap1_v131.frm", "gy1_v121.frm", "gy1_v130.frm"]
simple_filenames = [f+'.part1' for f in firmwares] + [f+'.part2' for f in firmwares]
filenames = [os.path.sep.join(['firmwares', 'parts', f]) for f in simple_filenames]
decoded_filenames = [os.path.sep.join(['firmwares', 'unxor', f+'.unxor.3']) for f in simple_filenames]

firmwares_parts = dict([(os.path.basename(fname),FirmwarePart(fname)) for fname in filenames])
decoded_firmwares_parts = dict([(os.path.basename(fname)[:-8],DecodedFirmwarePart(fname)) for fname in decoded_filenames])

keyfilename = os.path.sep.join(['firmwares', 'keys', 'consensus.pkeys'])
key_manager.add_keys_from_file('consensus', keyfilename)
kt_attack_keys = key_manager.get_bitview('consensus')
try:
    keyfilename = os.path.sep.join(['firmwares', 'keys', 'bitgen-attack.pkeys'])
    key_manager.add_keys_from_file('generated', keyfilename)
except IOError as e:
    pass

In [3]:
# Part 1 file contents
# the last few bytes are the filenames, offset and size
# last file is 0x390c00-> 0x00391ab8 (0xeb8)
import collections
starts = collections.defaultdict(list)
for fname, part in decoded_firmwares_parts.items():
    if 'part1' not in fname:
        continue
    files = part.read_file_list()
    # start is offseted by 0xa00.
    print '%s - %d files - total 0x%x bytes' % (fname, len(files), part.info_files[0][0])
    for name, start, size, u2, u3, u4 in files:
        print "%s \tstart:%0.8x size:%0.8x u2:%0.8x u3:%0.8x u4:%0.8x" %(name, start, size, u2, u3, u4)
        starts[name].append(hex(start))
    print ''

gy1_v130.frm.part1 - 133 files - total 0x393580 bytes
A:/fix/Sys_F.bin 	start:00000200 size:00000100 u2:48346418 u3:48346418 u4:48346418
A:/cpu1/tst.bin 	start:00000400 size:00009220 u2:48346344 u3:48346344 u4:48346344
A:/rosetta/RosList.bin 	start:00009800 size:00000144 u2:4834638c u3:4834638c u4:4834638c
A:/rosetta/CDcB.bin 	start:00009a00 size:00007354 u2:483462ed u3:4834638e u4:4834638e
A:/rosetta/r90.bin 	start:00010e00 size:00007354 u2:483462ed u3:4834638e u4:4834638e
A:/rosetta/CDcA.bin 	start:00018200 size:00007354 u2:483462ed u3:4834638e u4:4834638e
A:/rosetta/alpha.bin 	start:0001f600 size:00007354 u2:483462ed u3:4834638e u4:4834638e
A:/rosetta/tri.bin 	start:00026a00 size:00007800 u2:483462ed u3:4834638e u4:4834638e
A:/rosetta/afn1.bin 	start:0002e400 size:00008b80 u2:483462ed u3:4834638e u4:4834638e
A:/rosetta/afn0_m.bin 	start:00037000 size:00054600 u2:483462ed u3:4834638e u4:4834638e
A:/rosetta/afn0_s.bin 	start:0008b800 size:00009600 u2:483462ee u3:4834638e u4:4834638e
A

In [8]:
'''
ap1_v131.frm.part1 - 120 files - total 0x2a5240 bytes
A:/fix/Sys_F.bin 	start:00000200 size:00000100 u2:454e5126 u3:454e5126 u4:454e5126
A:/cpu1/tst.bin 	start:00000400 size:0000389c u2:454e5065 u3:454e5065 u4:454e5065
'''
# Extract files
import hashlib
hashes = collections.defaultdict(set)
hashes2 = collections.defaultdict(set)
for fname, part in decoded_firmwares_parts.items():
    if 'part1' not in fname:
        continue
    folder = os.path.sep.join(['firmwares','extracted',fname+'.d'])
    part = decoded_firmwares_parts[fname]
    files = part.read_file_list()
    # start is offseted by 0xa00.
    print '%s - %d files - total 0x%x bytes' % (fname, len(files), part.info_files[0][0])
    #for name, start, size, u2, u3, u4 in files:
    #    print "%s \tstart:%0.8x size:%0.8x u2:%0.8x u3:%0.8x u4:%0.8x" %(name, start, size, u2, u3, u4)
    #    starts[name].append(hex(start))
    i = 0
    for my_filename, _start, _size, _, _, _ in files:
        file_sub = my_filename.split('/')
        filename = file_sub[-1]
        subfolder = folder
        if len(file_sub) > 2:
            # mkdir subfolder
            subfolder = os.path.sep.join([folder,file_sub[1]])
            if not os.access(subfolder, os.F_OK):
                os.mkdir(subfolder)
        # extract file from 0xa00 or 0x800 ?
        start = part.HEADER_SIZE + 0x200 + _start
        end = start + _size
        data = part.data[start:end]
        fullfilename = os.path.sep.join([subfolder, filename])
        open(fullfilename, 'wb').write(data)
        # get hash
        hasher = hashlib.sha256()
        hasher.update(data)
        h = binascii.hexlify(hasher.digest())
        hashes[filename].add(h)
        hashes2[h].add(fullfilename)
        i+=1
    print '%d files written' % i

gy1_v130.frm.part1 - 133 files - total 0x393580 bytes
133 files written
gy1_v121.frm.part1 - 133 files - total 0x393580 bytes
133 files written
ap1_v130.frm.part1 - 120 files - total 0x2a5240 bytes
120 files written
ap1_v110.frm.part1 - 120 files - total 0x2a5240 bytes
120 files written
ap1_v131.frm.part1 - 120 files - total 0x2a5240 bytes
120 files written


In [10]:
for filename, hashs in hashes.items():
    if len(hashs) > 1:
        print filename
        for h in hashs:
            print '\t',h
            for fname in hashes2[h]:
                print '\t\t', fname 

# find similar files at different offset. Hoping for bindiff->xor keys perfection ?
for filename, startlist in starts.items():
    options = set(startlist)
    if len(options) == 1:
        continue
    print filename, options
                            

Sensor_A.bin
	bc755d5980b6e7132b91c62ce895e6ba5ceb210fc59b15caf22ec436cc0fb6df
		firmwares/extracted/ap1_v131.frm.part1.d/adj/Sensor_A.bin
		firmwares/extracted/ap1_v110.frm.part1.d/adj/Sensor_A.bin
		firmwares/extracted/ap1_v130.frm.part1.d/adj/Sensor_A.bin
	ede43c32d4f3f2cf596ccb430d0709650cd04791ebb994f212ad118e75f38d16
		firmwares/extracted/gy1_v121.frm.part1.d/adj/Sensor_A.bin
		firmwares/extracted/gy1_v130.frm.part1.d/adj/Sensor_A.bin
afn_ADcB.bin
	dd6a53b3543f2140dd7a4c7ce80711fa67501caa5d450e1022227ac54b626e3d
		firmwares/extracted/gy1_v130.frm.part1.d/isp/afn_ADcB.bin
		firmwares/extracted/gy1_v121.frm.part1.d/isp/afn_ADcB.bin
	88436a9f5a95ae79dfb1a0a91302d58a659b638a0ec067848d6361a9ab42acc2
		firmwares/extracted/ap1_v130.frm.part1.d/isp/afn_ADcB.bin
		firmwares/extracted/ap1_v131.frm.part1.d/isp/afn_ADcB.bin
		firmwares/extracted/ap1_v110.frm.part1.d/isp/afn_ADcB.bin
command.txt
	bfb04d79da58e01fb65490f67695f39c41843077ce139fe037d861b75b096299
		firmwares/extracted/ap1_v131.f